In [3]:
import numpy as np
import simpy 
from scipy.stats import expon

In [11]:
np.random.seed(42)

def arrival(env,servers, lambdas ,S,G):
    while True:
        dt = expon.rvs(scale = 0.5)
        arrival = env.timeout(dt)
        yield arrival
        S.append(env.now)
        env.process(service(env,servers,lambdas,S,G))
    
def service(env, servers,lambdas, S,G):
    lens = np.array([len(server.queue) + server.count for server in servers])
    ind = np.argmin(lens)
    rqt = servers[ind].request()
    yield rqt
    dt = expon.rvs(scale = 1/lambdas[ind])
    yield env.timeout(dt)
    servers[ind].release(rqt)
    
    G.append(env.now)
    
    
def simulate(n):
    S = []
    G = []
    env = simpy.Environment()
    servers = [simpy.Resource(env,capacity = 1) for i in range(0,n)]
    lambdas = [1,4,3]
    env.process(arrival(env,servers,lambdas,S,G))
    env.run(until=3)
    return S,G


simulate(3)

([0.23463404498842955,
  1.7396947604471902,
  2.196166037335167,
  2.280964183292469],
 [1.5513797385338788, 1.9093196309095364, 2.2560048059438476])